In [1]:
%matplotlib inline


Creating extensions using numpy and scipy
=========================================
**Author**: `Adam Paszke` [https://github.com/apaszke](https://github.com/apaszke)

**Updated by**: `Adam Dziedzic` [https://github.com/adam-dziedzic](https://github.com/adam-dziedzic)

In this tutorial, we shall go through two tasks:

1. Create a neural network layer with no parameters.

    -  This calls into **numpy** as part of it’s implementation

2. Create a neural network layer that has learnable weights

    -  This calls into **SciPy** as part of it’s implementation



In [2]:
import torch
from torch.autograd import Function

Parameter-less example
----------------------

This layer doesn’t particularly do anything useful or mathematically
correct.

It is aptly named BadFFTFunction

**Layer Implementation**



In [111]:
from numpy.fft import rfft2, irfft2

class BadFFTFunction(Function):
    @staticmethod
    def forward(self, input):
        numpy_input = input.detach().numpy()
        result = abs(rfft2(numpy_input))
        return input.new(result)

    @staticmethod
    def backward(self, grad_output):
        numpy_go = grad_output.numpy()
        result = irfft2(numpy_go)
        return grad_output.new(result)

# since this layer does not have any parameters, we can
# simply declare this as a function, rather than as an nn.Module class

def incorrect_fft(input):
    return BadFFTFunction()(input)


**Example usage of the created layer:**



In [112]:
input = torch.randn(8, 8, requires_grad=True)
print("the input to fft forward pass: ", input)
result = incorrect_fft(input)
print("result of forward fft: ", result)
result.backward(torch.randn(result.size()))
print("Gradient for the input: ", input.grad)
print("size of the gradient: ", input.grad.size())


the input to fft forward pass:  tensor([[-0.3340,  2.1187, -1.3309,  0.6148, -0.7633, -0.7805,  0.4434,
         -0.6965],
        [ 0.1043,  0.3583,  1.1323, -0.9970, -0.0019,  0.6787,  0.4112,
          1.5120],
        [-0.2142, -3.1594, -0.6444, -0.4585,  0.9635, -2.1726,  0.0333,
         -0.8942],
        [-0.3636, -0.0730, -0.3926,  0.6311,  0.8769,  1.0458, -1.3366,
         -0.8047],
        [-0.0504,  0.6493,  0.1017, -2.1825, -0.5158, -0.5393, -0.8690,
         -1.3570],
        [-0.9512, -1.2126,  1.6024, -0.6277, -2.2618,  0.8980,  0.0194,
         -0.5690],
        [-0.1116, -0.7792, -0.8842, -0.2721,  0.9937,  0.4120,  2.3265,
         -1.5192],
        [-0.9701,  0.0215, -0.5756,  1.4675, -1.9559, -0.5680,  0.7020,
         -0.0408]])


TypeError: forward() missing 1 required positional argument: 'input'

In [107]:
from torch.autograd import gradcheck

# gradcheck takes a tuple of tensors as input, check if your gradient
# evaluated with these tensors are close enough to numerical
# approximations and returns True if they all fulfill this condition

class FFTModule(Module):
    def __init__(self):
        super(FFTModule, self).__init__()

    def forward(self, input):
        return BadFFTFunction.apply(input)

fftModule = FFTModule()

input = [torch.randn(20, 20, dtype=torch.double, requires_grad=True)]
# print("input: ", input)
test = gradcheck(fftModule, input, eps=1e-6, atol=1e-4)
print("Are the gradients correct: ", test)

the input to fft forward pass:  tensor([[-0.3340,  2.1187, -1.3309,  0.6148, -0.7633, -0.7805,  0.4434,
         -0.6965],
        [ 0.1043,  0.3583,  1.1323, -0.9970, -0.0019,  0.6787,  0.4112,
          1.5120],
        [-0.2142, -3.1594, -0.6444, -0.4585,  0.9635, -2.1726,  0.0333,
         -0.8942],
        [-0.3636, -0.0730, -0.3926,  0.6311,  0.8769,  1.0458, -1.3366,
         -0.8047],
        [-0.0504,  0.6493,  0.1017, -2.1825, -0.5158, -0.5393, -0.8690,
         -1.3570],
        [-0.9512, -1.2126,  1.6024, -0.6277, -2.2618,  0.8980,  0.0194,
         -0.5690],
        [-0.1116, -0.7792, -0.8842, -0.2721,  0.9937,  0.4120,  2.3265,
         -1.5192],
        [-0.9701,  0.0215, -0.5756,  1.4675, -1.9559, -0.5680,  0.7020,
         -0.0408]])


TypeError: forward() missing 1 required positional argument: 'input'

Parametrized example
--------------------

An implementation of a layer with learnable weights, where cross-correlation has a learnable kernel.

In deep learning literature, it’s confusingly referred to as convolution while the actual operation is cross-correlation (the only difference is that filter is flipped for convolution, which is not the case for cross-correlation).

The backward pass computes the gradient wrt the input and the gradient wrt the
filter.

In [82]:
from numpy import flip
import numpy as np
from scipy.signal import correlate2d
from torch.nn.modules.module import Module
from torch.nn.parameter import Parameter

class ScipyConv2dFunction(Function):
    @staticmethod
    def forward(ctx, input, filter, bias):
        # detach so we can cast to NumPy
        input, filter, bias = input.detach(), filter.detach(), bias.detach()  
        result = correlate2d(input.numpy(), filter.numpy(), mode='valid')
        result += bias.numpy()
        ctx.save_for_backward(input, filter, bias)
        return torch.from_numpy(result)

    @staticmethod
    def backward(ctx, grad_output):
        grad_output = grad_output.detach()
        input, filter, bias = ctx.saved_tensors
        grad_output = grad_output.numpy()
        grad_bias = np.sum(grad_output, keepdims=True)
        grad_input = correlate2d(grad_output, flip(flip(filter.numpy(), axis=0), axis=1), mode='full')
        grad_filter = correlate2d(input.numpy(), grad_output, mode='valid')
        return torch.from_numpy(grad_input), torch.from_numpy(grad_filter), torch.from_numpy(grad_bias)


class ScipyConv2d(Module):
    def __init__(self, kh, kw):
        super(ScipyConv2d, self).__init__()
        self.filter = Parameter(torch.randn(kh, kw))
        self.bias = Parameter(torch.randn(1, 1))

    def forward(self, input):
        return ScipyConv2dFunction.apply(input, self.filter, self.bias)


**Example usage:**



In [81]:
module = ScipyConv2d(3, 3)
print("Filter and bias: ", list(module.parameters()))
input = torch.randn(10, 10, requires_grad=True)
output = module(input)
print("Output from the convolution: ", output)
output.backward(torch.randn(8, 8))
print("Gradient for the input map: ", input.grad)

Filter and bias:  [Parameter containing:
tensor([[ 0.9778, -2.1926, -1.5007],
        [-0.8564, -0.1252,  0.1965],
        [-1.0147, -1.9099, -0.3765]]), Parameter containing:
tensor(1.00000e-02 *
       [[ 8.8882]])]
Output from the convolution:  tensor([[ 3.7728, -0.5321,  1.2976,  3.0419,  0.7295,  2.7042,  0.3629,
         -0.6140],
        [-7.2798, -4.4747, -2.0613,  5.4178,  3.8800,  3.5487,  3.0402,
          0.8574],
        [ 4.3338, -4.8571, -3.9630,  0.2303,  3.9771,  0.5293, -0.2377,
         -3.1850],
        [-8.4965, -2.0868,  4.7418,  4.5396,  0.4477,  2.1242,  1.7706,
          1.2795],
        [-3.0739, -2.5370, -2.3028,  7.2233,  6.8461,  3.8279, -2.4395,
         -5.5369],
        [-2.3000, -0.0067,  5.2505,  2.9312,  2.7599, -0.4494,  3.1907,
          1.4693],
        [ 6.3484,  5.1985,  1.9591,  7.1688,  4.7119,  4.7957, -0.1767,
         -5.4811],
        [-2.2898,  3.0240,  1.5137,  3.9723,  1.5269, -2.4496,  2.4694,
         -4.7401]])
Gradient for the input 

In [96]:
moduleConv = ScipyConv2d(3, 3)

input = [torch.randn(20, 20, dtype=torch.double, requires_grad=True)]
# print("input: ", input)
test = gradcheck(moduleConv, input, eps=1e-6, atol=1e-4)
print("Are the gradients correct: ", test)

Are the gradients correct:  True
